In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import collections
from wordcloud import WordCloud
from konlpy.tag import Twitter
from collections import Counter
from nltk.probability import FreqDist
from sklearn.preprocessing import QuantileTransformer


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings("ignore")

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.size'] = 14

In [2]:
# import scipy.sparse as spr

---
# 데이터 불러오기

In [3]:
# train (playlist)
train = pd.read_json('train.json')

#genre
genre_gn_all = pd.read_json('genre_gn_all.json', typ='series')
genre_gn_all = pd.DataFrame(genre_gn_all, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'})

# songs
song_meta = pd.read_json('song_meta.json')

# plylst 별 tag 유사도를 살펴보자
- 1) `train`, `train['tags']` 확인
- 2) `train['tags']` [ ] 제거
- 3) plylst 좋아요 5개 이상 받은 plylst의 태그 선별 (11만 개 -> 3만3천개)
- 4) Tfidf Vectorizing 사용

## 전처리

앞서 진행했던 전처리 방법 그대로 적용해서 전처리를 진행하겠습니다.  
위에서 설명이 쓰여져 있지만, 데이터 내부엔 list(dict()) 형태로 구성되어 있습니다. 또한 이 안에는 **문자열** 형태로 들어가있구요.  
이를 처리하기 위해서 ast의 literal_eval 함수를 사용합니다.   
> `AST 모듈`은 문법을 구조화 시켜주는 모듈    
python 에서 제공하는 기본 type 정도만 변환해주는 용도로 사용 가능   
`literal_eval`은 python 의 기본 자료형 정도만 evaluate 가 가능하도록 지원한다. eval 과 비교해 훨씬 엄격   
그 사용 용도가 eval 대비 훨씬 제한적     
literal_eval 의 용도 :  string을 자료형으로 구조화 시키는 것.

> `apply 함수` :    
numpy의 단일함수를 호출 시 axis=0, 1의 결과값이 같음.   
numpy의 집계함수를 호출 시 axis=0 (행), axis=1 (열)

> `lambda x 함수` : 

In [4]:
# train 정보 확인.
train.info()
train.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115071 entries, 0 to 115070
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   tags          115071 non-null  object
 1   id            115071 non-null  int64 
 2   plylst_title  115071 non-null  object
 3   songs         115071 non-null  object
 4   like_cnt      115071 non-null  int64 
 5   updt_date     115071 non-null  object
dtypes: int64(2), object(4)
memory usage: 5.3+ MB


,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000


In [31]:
# sorted_songs_train = train.sort_values(by=['updt_date'], ascending=False)
# sorted_songs_train

In [6]:
# train tags 값 확인
train['tags']

0                                                       [락]
1                                                  [추억, 회상]
2                                                 [까페, 잔잔한]
3         [연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...
4                                                      [댄스]
                                ...                        
115066                 [록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]
115067                                                 [일렉]
115068               [담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]
115069                        [잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]
115070                               [노래추천, 팝송추천, 팝송, 팝송모음]
Name: tags, Length: 115071, dtype: object

###### map과 apply
> map = 컬럼 하나의 행에만 적용   
apply = 모든 행에 대해 적용

In [7]:
# tags [ ] 제거한 column(tags_string) 생성
train['tags_string'] = train['tags'].apply(lambda x: ' '.join(x))
train.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,tags_string
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,락
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,추억 회상
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,까페 잔잔한
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,댄스


## train_tagup5 변수 선언 :
- tags_cnt 태그 개수 카운트 
- 태그가 5개 이상 달린 plylst으로 이뤄진 DF변수 생성
>TfidfVectorizer을 태그 기반으로 수행하려고 했지만 allocate 용량이 너무 커서 에러남.    
플레이리스트 좋아요를 5개 이상 받은 플레이리스트로 범위를 줄여봄    
그 결과 11만개에서 3만3천개로 줄여짐.   

In [39]:
train['tags_cnt'].sum()

476331

In [8]:
# tags_cnt 만들어서 태그 개수 카운트 
# train_tagup5 변수 선언 : 태그가 5개 이상 달린 plylst으로 이뤄진 DF변수 생성
train['tags_cnt'] = train['tags'].map(lambda x : len(x)) # tags string으로 변환
train_tagup5 = train.query('tags_cnt >5') # 태그가 5개 이상 달린 플레이리스트 선별
print(train_tagup5.shape) # 11만개에서 3만3천개로 크기 줄임.
train_tagup5.head()

(33476, 8)


,tags,id,plylst_title,songs,like_cnt,updt_date,tags_string,tags_cnt
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송,10
5,"[운동, 드라이브, Pop, 트로피컬하우스, 힐링, 기분전환, 2017, 팝, 트렌...",69252,2017 Pop Trend,"[418694, 222305, 96545, 135950, 304687, 457451...",435,2017-09-15 15:59:26.000,운동 드라이브 Pop 트로피컬하우스 힐링 기분전환 2017 팝 트렌드 일렉,10
6,"[짝사랑, 취향저격, 슬픔, 고백, 사랑, 이별]",45339,짝사랑..고백..사랑..이별..슬픔.. 감성을 자극하는곡들!,"[528130, 505036, 398947, 627363, 37748, 376358...",24,2016-10-24 21:30:17.000,짝사랑 취향저격 슬픔 고백 사랑 이별,6
8,"[일렉트로니카, 포크, 메탈, 락, 댄스, 인디]",70741,DANCING IN THE MOON-LIGHT .01,"[634861, 270738, 163936, 692209, 449477, 56342...",0,2019-11-30 21:17:59.000,일렉트로니카 포크 메탈 락 댄스 인디,6
9,"[록, Metal, 이일우, M에센셜, 메탈, Rock, 락]",10288,[록/메탈] Written by 이일우,"[613315, 439294, 230806, 497932, 113479, 93506...",7,2016-06-17 18:23:54.000,록 Metal 이일우 M에센셜 메탈 Rock 락,7


---
### TF-IDF 벡터화

전처리한 데이터를 TF-IDF 방법을 이용해 벡터로 변환.  

> `단어 문서 행렬`(term-document matrix: 이하 `TDM`)   
`tfidf` 뒤에 `.get_feature_names()` 메써드를 부르면 TDM에 사용된 feature 단어 목록을 볼 수 있다.


In [9]:
tfidf_vector = TfidfVectorizer() 

# tfidf에 fit
tfidf_matrix = tfidf_vector.fit_transform(train_tagup5['tags_string']).toarray() 

# TDM에 사용된 feature 단어 목록을 볼 수 있다.
tfidf_matrix_feature = tfidf_vector.get_feature_names() 

In [10]:
tfidf_matrix # 벡터화된 태그(tags_cnt 5 이상 plylst)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
tfidf_matrix_feature # tags_cnt 5 이상 tags list

['00',
 '007',
 '007시리즈',
 '00s',
 '00년',
 '00년대',
 '00년대노래',
 '00년대발라드',
 '01',
 '015b',
 '02',
 '03',
 '03월',
 '04',
 '05',
 '0515',
 '06',
 '07',
 '0channel',
 '0개국어',
 '0살',
 '0시',
 '10',
 '100',
 '100000',
 '1000만관객동원',
 '100곡',
 '100위',
 '101',
 '10cm',
 '10곡',
 '10년간의_기록',
 '10년대',
 '10년전',
 '10대',
 '10대가10대에게',
 '10대감성',
 '10대들의우상',
 '10만개',
 '10분전',
 '10센치',
 '10월',
 '10월4주차',
 '10월음악',
 '10주',
 '10주년',
 '10초',
 '10화',
 '11',
 '110bpm',
 '11곡',
 '11월',
 '11월11일',
 '11월1일',
 '11월1주차',
 '11월3주차',
 '11월4주차',
 '11월_2주차',
 '11월_3주차',
 '11월_컴백',
 '11월넷째주',
 '11월둘째주',
 '11일',
 '11화',
 '12',
 '1218',
 '1225',
 '1248',
 '128bpm',
 '12월',
 '12월1주차',
 '12월25일',
 '12월2주차',
 '12월3주차',
 '12월_2주차',
 '12월둘째주',
 '12월셋째주',
 '12화',
 '13',
 '13명의',
 '13월',
 '13학번',
 '13화',
 '14',
 '1415',
 '1485',
 '14화',
 '14회',
 '15',
 '16',
 '16강',
 '16년_12월',
 '17년_10월',
 '17년_1월',
 '17년_3월',
 '17년_5월',
 '17주년',
 '18',
 '1855',
 '18번',
 '19',
 '1900년',
 '190113',
 '191229',
 '191231',
 '1920',
 '1930',
 '1930

In [12]:
# 행 = plylst_title, 열 = (tags_cnt > 5)인 tags
tfidf_matrix = pd.DataFrame(tfidf_matrix, columns=tfidf_matrix_feature, index = train_tagup5['plylst_title']) # 열 = 장르이름 | 행 = 영화이름
print(tfidf_matrix.shape) # tfidf_matrix 형태 : 행 = 태그의 개수, 열 = vector의 수
tfidf_matrix.head()

(33476, 21121)


,00,007,007시리즈,00s,00년,00년대,00년대노래,00년대발라드,01,015b,...,힙합페스티벌,힙합플레이리스트,힙합플레이야,힙합플레이야2018,힙합홍수,힙해,힙힙힙,힛뎀포크,힛뎀폭,힛뎀폭스
plylst_title,,,,,,,,,,,,,,,,,,,,,
크리스마스 분위기에 흠뻑 취하고 싶을때,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017 Pop Trend,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
짝사랑..고백..사랑..이별..슬픔.. 감성을 자극하는곡들!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DANCING IN THE MOON-LIGHT .01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
[록/메탈] Written by 이일우,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 유사도 구하기

이렇게 만들어진 `tf-idf vector`를 `코사인 유사도`를 활용해서 유사도 값을 구해줍니다.  
이렇게 하면 `plylst`의 개수(n)만큼 n x n의 matirx 형태가 나오게 됩니다.

In [13]:
# 코사인 유사도 구하기
%%time
cosine_sim = cosine_similarity(tfidf_matrix)

Wall time: 2min 43s


In [14]:
cosine_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.05918404],
       [0.        , 0.        , 1.        , ..., 0.        , 0.04024484,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.04024484, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.05918404, 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [15]:
cosine_sim.shape

(33476, 33476)

In [16]:
cosine_sim_df = pd.DataFrame(cosine_sim, index = train_tagup5['plylst_title'], columns = train_tagup5['plylst_title']) # 행 열 모두 영화 이름
print(cosine_sim_df.shape)
cosine_sim_df.head()

(33476, 33476)


plylst_title,크리스마스 분위기에 흠뻑 취하고 싶을때,2017 Pop Trend,짝사랑..고백..사랑..이별..슬픔.. 감성을 자극하는곡들!,DANCING IN THE MOON-LIGHT .01,[록/메탈] Written by 이일우,"노래로 의지를 불태우자! ""1일1다짐"" st용 프로필뮤직",트렌디하고 그루브한 힙합/알앤비 MUSIC,조용히 맥주 한잔 할때_새벽감성,"지친 하루 끝, 힐링이 필요한 당신에게 추천하는 인디곡",사랑스러운 남녀듀엣,...,취향저격 나만의 팝송 GET!!,사상 최대 규모의 Tomorrowland 2017이 기대되는 이유,장르상관없이 본인이 좋아하는노래들,"상쾌한 아침과 드라이브, 신나는 노래",4.일렉_ComeTogether!!!_custom.GOGR,Let's Groove 느낌있는 감성힙합,#기분업! 에너지필~ 사운드굿! 해외 EDM/일렉 선곡#2019년9월#,METAL E'SM #2,#1. 눈물이 앞을 가리는 나의_이야기,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP
plylst_title,,,,,,,,,,,,,,,,,,,,,
크리스마스 분위기에 흠뻑 취하고 싶을때,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017 Pop Trend,0.0,1.0,0.0,0.000000,0.000000,0.014587,0.032181,0.0,0.000000,0.000000,...,0.148364,0.0,0.047600,0.22953,0.141224,0.035407,0.129860,0.000000,0.000000,0.059184
짝사랑..고백..사랑..이별..슬픔.. 감성을 자극하는곡들!,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.162357,...,0.123700,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.040245,0.000000
DANCING IN THE MOON-LIGHT .01,0.0,0.0,0.0,1.000000,0.180306,0.000000,0.000000,0.0,0.127985,0.000000,...,0.000000,0.0,0.094858,0.00000,0.000000,0.000000,0.147023,0.167751,0.000000,0.000000
[록/메탈] Written by 이일우,0.0,0.0,0.0,0.180306,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.109440,0.000000,0.000000


In [17]:
recom_idx = cosine_sim_df.loc[:, '크리스마스 분위기에 흠뻑 취하고 싶을때'] # title series 가져오기
print(recom_idx)

plylst_title
크리스마스 분위기에 흠뻑 취하고 싶을때                      1.0
2017 Pop Trend                             0.0
짝사랑..고백..사랑..이별..슬픔.. 감성을 자극하는곡들!          0.0
DANCING IN THE MOON-LIGHT .01              0.0
[록/메탈] Written by 이일우                      0.0
                                          ... 
Let's Groove 느낌있는 감성힙합                     0.0
#기분업! 에너지필~ 사운드굿! 해외 EDM/일렉 선곡#2019년9월#    0.0
METAL E'SM #2                              0.0
#1. 눈물이 앞을 가리는 나의_이야기                      0.0
퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP          0.0
Name: 크리스마스 분위기에 흠뻑 취하고 싶을때, Length: 33476, dtype: float64


In [18]:
recom_idx = cosine_sim_df.loc[:, '크리스마스 분위기에 흠뻑 취하고 싶을때'].values # title values 가져오기
print(recom_idx)
print(type(recom_idx))
print(recom_idx.shape)

[1. 0. 0. ... 0. 0. 0.]
<class 'numpy.ndarray'>
(33476,)


In [19]:
recom_idx = cosine_sim_df.loc[:, '크리스마스 분위기에 흠뻑 취하고 싶을때'].values.reshape(1, -1) # 
print(recom_idx)
print(type(recom_idx))
print(recom_idx.shape)

[[1. 0. 0. ... 0. 0. 0.]]
<class 'numpy.ndarray'>
(1, 33476)


In [20]:
recom_idx = cosine_sim_df.loc[:, '크리스마스 분위기에 흠뻑 취하고 싶을때'].values.reshape(1, -1).argsort()[:, ::-1] # 코사인 유사도가 높은 기준으로 내림차순 정렬
print(recom_idx)
print(type(recom_idx))
print(recom_idx.shape)

[[    0 12430 11139 ... 21831 21832 16737]]
<class 'numpy.ndarray'>
(1, 33476)


In [21]:
# '크리스마스 분위기에 흠뻑 취하고 싶을때'와 코사인 유사도가 높은 순으로 plylst 벡터 10개 array만들기
recom_idx = cosine_sim_df.loc[:, '크리스마스 분위기에 흠뻑 취하고 싶을때'].values.reshape(1, -1).argsort()[:, ::-1].flatten()[0:10] 
print(recom_idx)
print(type(recom_idx))
print(recom_idx.shape)

[    0 12430 11139 17267 24324 19049  9556  1335  8917 25833]
<class 'numpy.ndarray'>
(10,)


In [33]:
recom_ply_title = train_tagup5.iloc[recom_idx, :]
display(recom_ply_title)
print(type(recom_ply_title))

,tags,id,plylst_title,songs,like_cnt,updt_date,tags_string,tags_cnt
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송,10
42943,"[연말, 감성, 겨울, 따듯한, 여행, 휴식, 12월, 크리스마스, 겨울왕국, 겨울노래]",131849,따듯한 겨울 감성 노래 모음,"[448698, 253755, 547797, 144371, 326204, 55358...",33,2019-12-05 15:07:36.000,연말 감성 겨울 따듯한 여행 휴식 12월 크리스마스 겨울왕국 겨울노래,10
38514,"[연말, 캐럴, 분위기, 재즈, 겨울노래, 크리스마스, 추위]",136791,크리스마스에 나만 알고 싶은 분위기 갑 재즈 뮤직,"[329465, 642891, 691717, 536895, 472194, 53963...",479,2019-12-09 06:00:28.000,연말 캐럴 분위기 재즈 겨울노래 크리스마스 추위,7
59558,"[연말, 캐럴, 카페, 크리스마스노래, 첫눈, 겨울, 12월, 크리스마스, 겨울왕국...",143665,내 마음에 쏙! 크리스마스 겨울노래 모음,"[387859, 253755, 326204, 311414, 260220, 52173...",34,2019-12-05 15:02:25.000,연말 캐럴 카페 크리스마스노래 첫눈 겨울 12월 크리스마스 겨울왕국 겨울노래,10
83395,"[연말, 눈오는날, 캐럴, 겨울밤, 눈, 12월, 크리스마스]",118922,캐럴 말고 크리스마스에 들을 노래,"[640403, 388107, 400105, 608572, 588063, 59535...",5,2017-12-25 14:28:39.000,연말 눈오는날 캐럴 겨울밤 눈 12월 크리스마스,7
65614,"[겨울, 크리스마스캐롤, 캐럴, 크리스마스캐럴, 12월, 크리스마스, 캐롤]",124330,"12월에는 크리스마스 캐롤을 들으며,","[387859, 226670, 326204, 695032, 55791, 587738...",12,2019-12-21 15:58:33.000,겨울 크리스마스캐롤 캐럴 크리스마스캐럴 12월 크리스마스 캐롤,7
33142,"[겨울, 눈, 산타, 겨울노래, 크리스마스, 캐롤, 크리스마스송, 추위]",4463,크리스마스에 꼭 듣게 되는 선물같은 노래,"[109102, 61771, 238699, 562860, 517006, 335077...",9,2018-12-24 23:11:34.000,겨울 눈 산타 겨울노래 크리스마스 캐롤 크리스마스송 추위,8
4924,"[눈내리는날, 연말, 겨울감성, 첫눈, 겨울, 겨울노래, 크리스마스, 겨울왕국]",128251,2019 누구나 좋아할만한 겨울 노래,"[567076, 62630, 71179, 379841, 244552, 600640,...",14,2019-12-25 03:58:26.000,눈내리는날 연말 겨울감성 첫눈 겨울 겨울노래 크리스마스 겨울왕국,8
30872,"[연말, 캐럴, 분위기, 겨울, 12월, 크리스마스, 캐롤, 겨울노래, 추위, Ch...",28640,힙한 팝캐롤과 함께하는 크리스마스❤,"[307928, 640547, 366881, 126848, 157248, 65187...",13,2019-12-26 11:01:14.000,연말 캐럴 분위기 겨울 12월 크리스마스 캐롤 겨울노래 추위 Christmas,10
88751,"[연말, 카페, 아이유, OST, 겨울노래, 크리스마스, 겨울왕국, 디즈니, 띵곡,...",36527,"2019 겨울, 장르불문 핫 트렌드 띵곡","[385871, 667631, 420672, 497865, 387859, 44869...",32,2019-12-05 15:15:18.000,연말 카페 아이유 OST 겨울노래 크리스마스 겨울왕국 디즈니 띵곡 유산슬,10


<class 'pandas.core.frame.DataFrame'>


In [34]:
recom_ply_title = train_tagup5.iloc[recom_idx, :].plylst_title
print(type(recom_ply_title))
print(recom_ply_title)

<class 'pandas.core.series.Series'>
3              크리스마스 분위기에 흠뻑 취하고 싶을때
42943                따듯한 겨울 감성 노래 모음
38514    크리스마스에 나만 알고 싶은 분위기 갑 재즈 뮤직
59558         내 마음에 쏙! 크리스마스 겨울노래 모음
83395             캐럴 말고 크리스마스에 들을 노래
65614           12월에는 크리스마스 캐롤을 들으며,
33142         크리스마스에 꼭 듣게 되는 선물같은 노래
4924            2019 누구나 좋아할만한 겨울 노래
30872            힙한 팝캐롤과 함께하는 크리스마스❤
88751         2019 겨울, 장르불문 핫 트렌드 띵곡
Name: plylst_title, dtype: object


In [35]:
recom_ply_title = train_tagup5.iloc[recom_idx, :].plylst_title.values
print(type(recom_ply_title))
print(recom_ply_title)

<class 'numpy.ndarray'>
['크리스마스 분위기에 흠뻑 취하고 싶을때' '따듯한 겨울 감성 노래 모음' '크리스마스에 나만 알고 싶은 분위기 갑 재즈 뮤직'
 '내 마음에 쏙! 크리스마스 겨울노래 모음' '캐럴 말고 크리스마스에 들을 노래' '12월에는 크리스마스 캐롤을 들으며,'
 '크리스마스에 꼭 듣게 되는 선물같은 노래' '2019 누구나 좋아할만한 겨울 노래' '힙한 팝캐롤과 함께하는 크리스마스❤'
 '2019 겨울, 장르불문 핫 트렌드 띵곡']


In [36]:
recom_tag = train_tagup5.iloc[recom_idx, :]
display(recom_tag.head(2))

,tags,id,plylst_title,songs,like_cnt,updt_date,tags_string,tags_cnt
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송,10
42943,"[연말, 감성, 겨울, 따듯한, 여행, 휴식, 12월, 크리스마스, 겨울왕국, 겨울노래]",131849,따듯한 겨울 감성 노래 모음,"[448698, 253755, 547797, 144371, 326204, 55358...",33,2019-12-05 15:07:36.000,연말 감성 겨울 따듯한 여행 휴식 12월 크리스마스 겨울왕국 겨울노래,10


In [37]:
recom_tag = train_tagup5.iloc[recom_idx, :].tags_string
display(recom_tag)

3        연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송
42943               연말 감성 겨울 따듯한 여행 휴식 12월 크리스마스 겨울왕국 겨울노래
38514                           연말 캐럴 분위기 재즈 겨울노래 크리스마스 추위
59558           연말 캐럴 카페 크리스마스노래 첫눈 겨울 12월 크리스마스 겨울왕국 겨울노래
83395                           연말 눈오는날 캐럴 겨울밤 눈 12월 크리스마스
65614                   겨울 크리스마스캐롤 캐럴 크리스마스캐럴 12월 크리스마스 캐롤
33142                      겨울 눈 산타 겨울노래 크리스마스 캐롤 크리스마스송 추위
4924                   눈내리는날 연말 겨울감성 첫눈 겨울 겨울노래 크리스마스 겨울왕국
30872          연말 캐럴 분위기 겨울 12월 크리스마스 캐롤 겨울노래 추위 Christmas
88751             연말 카페 아이유 OST 겨울노래 크리스마스 겨울왕국 디즈니 띵곡 유산슬
Name: tags_string, dtype: object

In [38]:
recom_tag = train_tagup5.iloc[recom_idx, :].tags_string.values
display(recom_tag)

array(['연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송',
       '연말 감성 겨울 따듯한 여행 휴식 12월 크리스마스 겨울왕국 겨울노래',
       '연말 캐럴 분위기 재즈 겨울노래 크리스마스 추위',
       '연말 캐럴 카페 크리스마스노래 첫눈 겨울 12월 크리스마스 겨울왕국 겨울노래',
       '연말 눈오는날 캐럴 겨울밤 눈 12월 크리스마스', '겨울 크리스마스캐롤 캐럴 크리스마스캐럴 12월 크리스마스 캐롤',
       '겨울 눈 산타 겨울노래 크리스마스 캐롤 크리스마스송 추위',
       '눈내리는날 연말 겨울감성 첫눈 겨울 겨울노래 크리스마스 겨울왕국',
       '연말 캐럴 분위기 겨울 12월 크리스마스 캐롤 겨울노래 추위 Christmas',
       '연말 카페 아이유 OST 겨울노래 크리스마스 겨울왕국 디즈니 띵곡 유산슬'], dtype=object)

In [28]:
target_ply_title = np.full(len(range(10)), '크리스마스 분위기에 흠뻑 취하고 싶을때')
target_ply_title

array(['크리스마스 분위기에 흠뻑 취하고 싶을때', '크리스마스 분위기에 흠뻑 취하고 싶을때',
       '크리스마스 분위기에 흠뻑 취하고 싶을때', '크리스마스 분위기에 흠뻑 취하고 싶을때',
       '크리스마스 분위기에 흠뻑 취하고 싶을때', '크리스마스 분위기에 흠뻑 취하고 싶을때',
       '크리스마스 분위기에 흠뻑 취하고 싶을때', '크리스마스 분위기에 흠뻑 취하고 싶을때',
       '크리스마스 분위기에 흠뻑 취하고 싶을때', '크리스마스 분위기에 흠뻑 취하고 싶을때'], dtype='<U21')

In [29]:
target_tags_list = np.full(len(range(10)), train[train.plylst_title == '크리스마스 분위기에 흠뻑 취하고 싶을때'].tags_string.values)
target_tags_list

array(['연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송',
       '연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송',
       '연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송',
       '연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송',
       '연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송',
       '연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송',
       '연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송',
       '연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송',
       '연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송',
       '연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송'], dtype=object)

---
# word_2_vec으로 도전

In [30]:
from konlpy.tag import Okt  
okt=Okt()